In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
# external imports
import numpy as np
import matplotlib.pyplot as plt

# internal imports
from spp.pwa_systems import ShortestPathRegulator
from ball_and_paddle_parameters import K, Z, cost_matrices
from ball_and_paddle_dynamics import pwa

In [ ]:
# initial state
z1 = np.array([
    0., 0., np.pi,
    0., 0.,
    0., 0., 0.,
    0., 0.
])

# solve optimal control problem
reg = ShortestPathRegulator(pwa, K + 1, z1, Z, cost_matrices, relaxation=1)
sol = reg.solve()
spp_cost = sol.spp.cost
spp_time = sol.spp.time
print('Cost:', sol.spp.cost)
print('Solve time:', sol.spp.time)

In [ ]:
plt.figure(figsize=(6, 2))

ms_legend = [
    'Slide left on paddle',
    'Stick/roll on paddle',
    'Slide right on paddle',
    'No contact',
    'Slide left on ceiling',
    'Stick/roll on ceiling',
    'Slide right on ceiling'
]
ms_map = {0: 3, 1: 1, 2: 2, 3: 0, 4: 5, 5: 6, 6: 4}

# plot mode sequence
for k in range(K):
    for i in range(pwa.nm):
        plt.scatter(k + 1, ms_map[i], ec='k', fc='w')
        E = reg.spp.graph.outgoing_edges((k + 1, i))[1]
        alpha = sum(sol.spp.primal.phi[e] for e in E)
        plt.scatter(k + 1, ms_map[i], ec='k', fc='b', alpha=alpha)
        
        for j in range(pwa.nm):
            edge = ((k, i), (k + 1, j))
            if edge in reg.spp.graph.edges:
                alpha = np.abs(sol.spp.primal.phi[reg.spp.graph.edge_index(edge)])
                plt.plot((k, k + 1), (ms_map[i], ms_map[j]), c='k', alpha=alpha, zorder=-1, lw=3)
            
plt.xticks(np.arange(11) * 2)
plt.xlim([0, 21])
plt.xlabel('Time step')
plt.yticks(range(7), ms_legend)
# plt.savefig('ball_and_paddle_ms_spp.pdf', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(6, 2))

# plot mode sequence
ms = [1, 1, 0, 5, 0, 0, 3, 1, 1, 1, 1, 1, 1, 0, 6, 0, 0, 0, 1, 1]
for k in range(K):
    for i in range(pwa.nm):
        plt.scatter(k + 1, ms_map[i], ec='k', fc='w')
        if ms[k] == i:
            plt.scatter(k + 1, ms_map[i], ec='k', fc='b')
            
plt.xticks(np.arange(11) * 2)
plt.xlim([0, 21])
plt.xlabel('Time step')
plt.yticks(range(7), ms_legend)
# plt.savefig('ball_and_paddle_ms.pdf', bbox_inches='tight')

In [ ]:
def solve_fxed_ms(ms):
    
    edges = [(0, (1, ms[0]))]
    for t in range(K - 1):
        edges.append(((t+1, ms[t]), (t+2, ms[t+1])))
    edges.append(((K, ms[-1]), K + 1))
    
    reg = ShortestPathRegulator(pwa, K + 1, z1, Z, cost_matrices, relaxation=1)
    for k in reg.spp.graph.edge_indices(edges):
        reg.spp.prog.AddLinearConstraint(reg.spp.vars.phi[k] == 1)
    sol = reg.solve()
    
    return sol.spp.cost

In [ ]:
def randomized_rounding(edges, phi):
    
    def add_mode(ms, E_v):
        phi_v = np.abs(np.array([phi[i] for i in E_v]))
        i = np.random.choice(E_v, p=phi_v/sum(phi_v))
        ms.append(edges[i][1][1])
        
    ms = []
    E_s = [i for i, e in enumerate(edges) if e[0] == 0]
    add_mode(ms, E_s)
    for k in range(1, K):
        E_v = [i for i, e in enumerate(edges) if e[0] == (k, ms[-1])]
        add_mode(ms, E_v)
        
    return(ms)

In [ ]:
np.random.seed(0)
for i in range(100):
    ms = randomized_rounding(reg.spp.graph.edges, sol.spp.primal.phi)
    print(ms)
    print(solve_fxed_ms(ms))

In [ ]:
optimal_cost_one = 63.16544
optimal_cost_two = 37.20787
ms_one = [1, 1, 0, 5, 0, 0, 0, 3, 1, 1, 1, 1, 1, 0, 6, 0, 0, 0, 1, 1]
ms_two = [1, 1, 0, 5, 0, 0, 3, 1, 1, 1, 1, 1, 1, 0, 6, 0, 0, 0, 1, 1]

In [ ]:
relaxation = 1
reg = ShortestPathRegulator(pwa, K + 1, z1, Z, cost_matrices, relaxation=relaxation)
optimal_ms = ms_two
edges = [(0, (1, optimal_ms[0]))]
for t in range(len(optimal_ms) - 1):
    edges.append(((t+1, optimal_ms[t]), (t+2, optimal_ms[t+1])))
edges.append(((len(optimal_ms), optimal_ms[-1]), len(optimal_ms) + 1))
for k in reg.spp.graph.edge_indices(edges):
    reg.spp.prog.AddLinearConstraint(reg.spp.vars.phi[k] == 1)
sol = reg.solve()
print('Cost:', sol.spp.cost)
print('Solve time:', sol.spp.time)

In [ ]:
# solved with the convex-hull formulation
times = np.array([0,0,0,0,2,4,4,4,4,4,5,5,5,5,6,10,10,11,11,15,15,20,21,21,23,23,25,27,27,27,30,35,40,45,51,55,61,73,75,81,86,91,97,101,108,112,135,169,174,178,182,186,190,198,202,206,213,217,220,228,231,235,243,247,251,258,262,265,272,276,280,287,291,295,302,306,313,317,321,328,331,335,342,345,352,356,363,366,373,376,381,388,391,395,402,405,412,415,422,427,430,437,441,448,452,455,462,466,473,477,480,487,490,497,500,507,510,517,520,527,530,536,542,546,552,556,562,562,562,580,585,590,597,602,602,602,605,611,616,622,625,632,635,641,645,651,659,662,665,670,676,681,686,691,696,701,706,711,715,722,726,730,735,741,745,750,756,760,766,770,775,781,788,790,795,800,805,811,815,820,825,830,836,840,845,850,857,860,865,870,878,880,885,891,896,900,907,910,915,921,928,930,935,940,947,952,956,961,966,971,976,981,986,991,996,1000,1005,1010,1015,1020,1026,1030,1035,1040,1045,1050,1055,1062,1067,1071,1076,1081,1085,1090,1096,1101,1106,1110,1115,1120,1125,1130,1135,1141,1146,1151,1156,1160,1165,1170,1175,1180,1185,1190,1195,1203,1205,1210,1216,1221,1226,1230,1235,1240,1245,1250,1255,1260,1266,1271,1276,1281,1286,1290,1295,1300,1306,1310,1315,1320,1326,1331,1336,1340,1345,1350,1355,1360,1365,1371,1375,1381,1386,1390,1396,1401,1405])
lower_bounds = np.array([2.81801,2.81801,2.81801,2.81801,2.81801, 3.09376,3.09376,3.09376,3.09376,3.09376,3.09376,3.09376,3.09376,3.09376,3.09376,3.11560,3.11560,3.11560,3.11560,3.20493,3.20493,3.55375,3.65367,3.65367,3.65367,3.65367,3.82467,3.96100,3.96100,3.96100,4.69987,6.69230,7.61162,8.36124,8.84296,9.15105,9.46058,9.54443,9.81345,10.20233,10.34595,10.47541,10.65123,10.76072,10.96584,11.07014,11.21587,11.23433,12.08938,12.14846,12.30274,12.54519,12.70072,12.99263,13.12808,13.19463,13.51718,13.66630,13.81658,14.05649,14.22308,14.35984,14.58593,14.74512,14.86645,15.10516,15.20846,15.31540,15.52373,15.61090,15.70907,15.98130,16.12300,16.27754,16.53021,16.66294,16.92657,17.02647,17.10411,17.33382,17.47666,17.56948,17.80192,17.90236,18.05868,18.17331,18.41627,18.53251,18.72178,18.79362,18.86138,19.09253,19.18367,19.30267,19.48653,19.60564,19.80684,19.92225,20.15445,20.33006,20.48942,20.78791,20.89363,21.10958,21.21018,21.30386,21.49412,21.64079,21.81629,21.99233,22.11351,22.39611,22.48970,22.78654,22.87051,23.19544,23.30435,23.54162,23.69132,23.95152,24.09308,24.31513,24.52629,24.59597,24.80124,24.93489,24.97806,24.97806,24.97806,24.98584,25.33343,25.52624,25.83360,26.01010,26.01010,26.03746,26.05843,26.38101,26.63961,26.84386,27.01003,27.36458,27.51735,27.83524,27.99582,28.24247,28.49456,28.61932,28.84866,29.10987,29.33487,29.57424,29.79910,29.96461,30.19951,30.38351,30.56935,30.66744,30.86299,31.04884,31.16584,31.24565,31.32267,31.41095,31.49174,31.57603,31.66036,31.70349,31.78805,31.83149,31.89958,31.94288,32.00438,32.03867,32.10279,32.16144,32.21898,32.25244,32.30455,32.35260,32.40344,32.46105,32.50685,32.53486,32.58507,32.62595,32.67007,32.71820,32.75178,32.79635,32.85452,32.87573,32.92019,32.96858,33.00452,33.02012,33.05830,33.05830,33.10079,33.13720,33.18012,33.19923,33.23819,33.26813,33.31574,33.31917,33.35730,33.38504,33.42019,33.46217,33.49474,33.52130,33.56963,33.60044,33.62206,33.65088,33.68518,33.71496,33.73723,33.77050,33.80663,33.83616,33.86897,33.88594,33.92654,33.96068,33.98972,34.01705,34.04721,34.06422,34.09354,34.12544,34.15115,34.19917,34.23227,34.26247,34.29972,34.33788,34.36952,34.40185,34.42943,34.46391,34.50433,34.53714,34.56751,34.60267,34.63715,34.66903,34.69713,34.73556,34.76519,34.79318,34.82766,34.85529,34.89431,34.93774,34.93902,34.96986,34.99248,35.02530,35.05623,35.08561,35.11901,35.15256,35.17863,35.21004,35.24742,35.28114,35.32356,35.35464,35.38778,35.41872,35.45350,35.49063,35.53362,35.56369,35.62311,35.66933,35.71481,35.76171,35.79472,35.83910,35.89211,35.94851,36.00642,36.07665,36.14004,36.21168,36.28207,36.36658,36.43938,36.56303,36.63873,36.73309,36.85612,37.00139,37.13511])
upper_bounds = np.array([np.nan,np.nan,np.nan,np.nan,np.nan,44.3551011,43.8134253,43.5643909,43.1017426,42.5875757,42.0287339,41.4759559,41.0164180,40.8938621,40.4992266,40.49923,39.2862119,38.8709574,38.6571645,38.65716,38.6295786,38.62958,38.2478595,38.2341117,38.2341095,38.0790615,38.07906,38.0778430,38.0768988,38.0358232,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,38.03582,37.6969333,37.3975373,37.39754,37.39754,37.39754,37.39754,37.39754,37.2401763,37.2078664,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787,37.20787])
optimal_cost = 37.20787

In [ ]:
plt.plot(times, [optimal_cost] * len(times), 'g:', label='Optimal cost')
plt.plot(times, upper_bounds, 'b-', label='Upper bound')
plt.plot(times, lower_bounds, 'b--', label='Lower bound')
plt.scatter(spp_time, spp_cost, marker='*', color='r', label='SPP relaxation')


plt.xlim(1e-2, times[-1])
plt.xlabel('Branch-and-bound time (s)')
plt.ylabel('Optimization cost')

plt.gca().set_xscale('log')
plt.grid()
plt.legend(loc=4)

In [ ]:
plt.figure(figsize=(6, 3))

# plot mode sequence
ms = [1, 1, 0, 5, 0, 0, 3, 1, 1, 1, 1, 1, 1, 0, 6, 0, 0, 0, 1, 1]
for k in range(K):
    for i in range(pwa.nm):
        plt.scatter(k + 1, i + 1, ec='k', fc='w')
        if ms[k] == i:
            plt.scatter(k + 1, i + 1, ec='k', fc='b')
            
plt.xticks([0, 5, 15, 20])
plt.xlim([0, 21])
plt.xlabel('Time step')
plt.ylabel('Contact mode')
plt.savefig('ball_and_paddle_ms.pdf', bbox_inches='tight')